In [1]:
import llm

In [2]:
source_code = """
 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_mol, atm2lnd_inst, photosyns_inst)
  !
  !! DESCRIPTION:
  ! evaluate the function
  ! f(ci)=ci - (ca - (1.37rb+1.65rs))*patm*an
  !
  ! remark:  I am attempting to maintain the original code structure, also
  ! considering one may be interested to output relevant variables for the
  ! photosynthesis model, I have decided to add these relevant variables to
  ! the relevant data types.
  !
  !!ARGUMENTS:
  REAL(KIND = r8), INTENT(IN) :: ci
  ! intracellular leaf CO2 (Pa)
  REAL(KIND = r8), INTENT(IN) :: lmr_z
  ! canopy layer: leaf maintenance respiration rate (umol CO2/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: par_z
  ! par absorbed per unit lai for canopy layer (w/m**2)
  REAL(KIND = r8), INTENT(IN) :: gb_mol
  ! leaf boundary layer conductance (umol H2O/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: je
  ! electron transport rate (umol electrons/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: cair
  ! Atmospheric CO2 partial pressure (Pa)
  REAL(KIND = r8), INTENT(IN) :: oair
  ! Atmospheric O2 partial pressure (Pa)
  REAL(KIND = r8), INTENT(IN) :: rh_can
  ! canopy air realtive humidity
  INTEGER, INTENT(IN) :: p, iv, c
  ! pft, vegetation type and column indexes
  REAL(KIND = r8), INTENT(OUT) :: fval
  ! return function of the value f(ci)
  REAL(KIND = r8), INTENT(OUT) :: gs_mol
  ! leaf stomatal conductance (umol H2O/m**2/s)
  TYPE(atm2lnd_type), INTENT(IN) :: atm2lnd_inst
  TYPE(photosyns_type), INTENT(INOUT) :: photosyns_inst
  !
  !local variables
  REAL(KIND = r8) :: ai
  ! intermediate co-limited photosynthesis (umol CO2/m**2/s)
  REAL(KIND = r8) :: cs
  ! CO2 partial pressure at leaf surface (Pa)
  REAL(KIND = r8) :: term
  ! intermediate in Medlyn stomatal model
  REAL(KIND = r8) :: aquad, bquad, cquad
  ! terms for quadratic equations
  REAL(KIND = r8) :: r1, r2
  ! roots of quadratic equation
  !------------------------------------------------------------------------------

  ASSOCIATE(forc_pbot => atm2lnd_inst % forc_pbot_downscaled_col, c3flag => photosyns_inst % c3flag_patch, ivt => patch % itype, medlynslope => pftcon % medlynslope, medlynintercept => pftcon % medlynintercept, stomatalcond_mtd => photosyns_inst % stomatalcond_mtd, ac => photosyns_inst % ac_patch, aj => photosyns_inst % aj_patch, ap => photosyns_inst % ap_patch, ag => photosyns_inst % ag_patch, an => photosyns_inst % an_patch, vcmax_z => photosyns_inst % vcmax_z_patch, cp => photosyns_inst % cp_patch, kc => photosyns_inst % kc_patch, ko => photosyns_inst % ko_patch, qe => photosyns_inst % qe_patch, tpu_z => photosyns_inst % tpu_z_patch, kp_z => photosyns_inst % kp_z_patch, bbb => photosyns_inst % bbb_patch, mbb => photosyns_inst % mbb_patch)
    ! Output: [real(r8) (:)   ]  atmospheric pressure (Pa)
      ! Output: [logical  (:)   ]  true if C3 and false if C4
      ! Input:  [integer  (:)   ]  patch vegetation type
      ! Input:  [real(r8) (:)   ]  Slope for Medlyn stomatal conductance model method
      ! Input:  [real(r8) (:)   ]  Intercept for Medlyn stomatal conductance model method
      ! Input:  [integer        ]  method type to use for stomatal conductance
      ! Output: [real(r8) (:,:) ]  Rubisco-limited gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  RuBP-limited gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  product-limited (C3) or CO2-limited (C4) gross photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  co-limited gross leaf photosynthesis (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  net leaf photosynthesis (umol CO2/m**2/s)
      ! Input:  [real(r8) (:,:) ]  maximum rate of carboxylation (umol co2/m**2/s)
      ! Output: [real(r8) (:)   ]  CO2 compensation point (Pa)
      ! Output: [real(r8) (:)   ]  Michaelis-Menten constant for CO2 (Pa)
      ! Output: [real(r8) (:)   ]  Michaelis-Menten constant for O2 (Pa)
      ! Output: [real(r8) (:)   ]  quantum efficiency, used only for C4 (mol CO2 / mol photons)
      ! Output: [real(r8) (:,:) ]  triose phosphate utilization rate (umol CO2/m**2/s)
      ! Output: [real(r8) (:,:) ]  initial slope of CO2 response curve (C4 plants)
      ! Output: [real(r8) (:)   ]  Ball-Berry minimum leaf conductance (umol H2O/m**2/s)
      ! Output: [real(r8) (:)   ]  Ball-Berry slope of conductance-photosynthesis relationship

      IF (c3flag(p)) THEN
      ! C3: Rubisco-limited photosynthesis
      ac(p, iv) = vcmax_z(p, iv) * MAX(ci - cp(p), 0._r8) / (ci + kc(p) * (1._r8 + oair / ko(p)))

      ! C3: RuBP-limited photosynthesis
      aj(p, iv) = je * MAX(ci - cp(p), 0._r8) / (4._r8 * ci + 8._r8 * cp(p))

      ! C3: Product-limited photosynthesis
      ap(p, iv) = 3._r8 * tpu_z(p, iv)

    ELSE

      ! C4: Rubisco-limited photosynthesis
      ac(p, iv) = vcmax_z(p, iv)

      ! C4: RuBP-limited photosynthesis
      aj(p, iv) = qe(p) * par_z * 4.6_r8

      ! C4: PEP carboxylase-limited (CO2-limited)
      ap(p, iv) = kp_z(p, iv) * MAX(ci, 0._r8) / forc_pbot(c)

    END IF

    ! Gross photosynthesis. First co-limit ac and aj. Then co-limit ap

    aquad = params_inst % theta_cj(ivt(p))
    bquad = - (ac(p, iv) + aj(p, iv))
    cquad = ac(p, iv) * aj(p, iv)
    CALL quadratic(aquad, bquad, cquad, r1, r2)
    ai = MIN(r1, r2)

    aquad = params_inst % theta_ip
    bquad = - (ai + ap(p, iv))
    cquad = ai * ap(p, iv)
    CALL quadratic(aquad, bquad, cquad, r1, r2)
    ag(p, iv) = MAX(0._r8, MIN(r1, r2))

    ! Net photosynthesis. Exit iteration if an < 0

    an(p, iv) = ag(p, iv) - lmr_z
    IF (an(p, iv) < 0._r8) THEN
      fval = 0._r8
      RETURN
    END IF
    ! Quadratic gs_mol calculation with an known. Valid for an >= 0.
    ! With an <= 0, then gs_mol = bbb or medlyn intercept
    cs = cair - 1.4_r8 / gb_mol * an(p, iv) * forc_pbot(c)
    cs = MAX(cs, max_cs)
    IF (stomatalcond_mtd == stomatalcond_mtd_medlyn2011) THEN
      term = 1.6_r8 * an(p, iv) / (cs / forc_pbot(c) * 1.E06_r8)
      aquad = 1.0_r8
      bquad = - (2.0 * (medlynintercept(patch % itype(p)) * 1.E-06_r8 + term) + (medlynslope(patch % itype(p)) * term) ** 2 / (gb_mol * 1.E-06_r8 * rh_can))
      cquad = medlynintercept(patch % itype(p)) * medlynintercept(patch % itype(p)) * 1.E-12_r8 + (2.0 * medlynintercept(patch % itype(p)) * 1.E-06_r8 + term * (1.0 - medlynslope(patch % itype(p)) * medlynslope(patch % itype(p)) / rh_can)) * term

      CALL quadratic(aquad, bquad, cquad, r1, r2)
      gs_mol = MAX(r1, r2) * 1.E06_r8
    ELSE IF (stomatalcond_mtd == stomatalcond_mtd_bb1987) THEN
      aquad = cs
      bquad = cs * (gb_mol - bbb(p)) - mbb(p) * an(p, iv) * forc_pbot(c)
      cquad = - gb_mol * (cs * bbb(p) + mbb(p) * an(p, iv) * forc_pbot(c) * rh_can)
      CALL quadratic(aquad, bquad, cquad, r1, r2)
      gs_mol = MAX(r1, r2)
    END IF




    ! Derive new estimate for ci

    fval = ci - cair + an(p, iv) * forc_pbot(c) * (1.4_r8 * gs_mol + 1.6_r8 * gb_mol) / (gb_mol * gs_mol)

  END ASSOCIATE

END SUBROUTINE ci_func
"""

In [3]:
fortran_unit_tests = llm._generate_fortran_unit_tests(source_code)

Generating unit tests in Fortran...
PROMPT: 
    Given fortran code, write unit tests using funit.

    Example:
    FORTRAN CODE:
    ```
    module fac
        implicit none
        
        contains

        recursive function factorial(n) result(fact)
            integer, intent(in) :: n
            integer :: fact

            if (n == 0) then
            fact = 1
            else
            fact = n * factorial(n - 1)
            end if
        end function factorial
    end module fac
    ```

    FORTRAN TESTS:
    ```
    @test
    subroutine test_fac()
        use funit

        @assertEqual(120, factorial(5), 'factorial(5)')
        @assertEqual(1, factorial(1), 'factorial(1)')
        @assertEqual(1, factorial(0), 'factorial(0)')

    end subroutine test_fac
    ```

    Your turn:
    FORTRAN CODE:
```

 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_

In [4]:
python_unit_tests = llm._translate_tests_to_python(fortran_unit_tests)

Translating unit tests to Python...
PROMPT: 
    Convert the following unit tests from Fortran to Python using pytest. No need to import the module under test. ```

!------------------------------------------------------------------------------
SUBROUTINE test_ci_func()
  USE funit

  ! Define input variables
  REAL(KIND = r8), PARAMETER :: ci = 400.0_r8
  REAL(KIND = r8), PARAMETER :: lmr_z = 1.0_r8
  REAL(KIND = r8), PARAMETER :: par_z = 100.0_r8
  REAL(KIND = r8), PARAMETER :: gb_mol = 0.1_r8
  REAL(KIND = r8), PARAMETER :: je = 10.0_r8
  REAL(KIND = r8), PARAMETER :: cair = 400.0_r8
  REAL(KIND = r8), PARAMETER :: oair = 210.0_r8
  REAL(KIND = r8), PARAMETER :: rh_can = 0.5_r8
  INTEGER, PARAMETER :: p = 1
  INTEGER, PARAMETER :: iv = 1
  INTEGER, PARAMETER :: c = 1

  ! Define expected output values
  REAL(KIND = r8), PARAMETER :: expected_fval = 0.0_r8
  REAL(KIND = r8), PARAMETER :: expected_gs_mol = 0.0_r8

  ! Define variables for actual output values
  REAL(KIND = r8) :: actu

In [5]:
# Generate a translated Python function
python_function = llm._translate_function_to_python(source_code)

Translating function to Python...
PROMPT: 
    Convert the following Fortran function to Python. ```

 !------------------------------------------------------------------------------
  SUBROUTINE ci_func(ci, fval, p, iv, c, gb_mol, je, cair, oair, lmr_z, par_z, rh_can, gs_mol, atm2lnd_inst, photosyns_inst)
  !
  !! DESCRIPTION:
  ! evaluate the function
  ! f(ci)=ci - (ca - (1.37rb+1.65rs))*patm*an
  !
  ! remark:  I am attempting to maintain the original code structure, also
  ! considering one may be interested to output relevant variables for the
  ! photosynthesis model, I have decided to add these relevant variables to
  ! the relevant data types.
  !
  !!ARGUMENTS:
  REAL(KIND = r8), INTENT(IN) :: ci
  ! intracellular leaf CO2 (Pa)
  REAL(KIND = r8), INTENT(IN) :: lmr_z
  ! canopy layer: leaf maintenance respiration rate (umol CO2/m**2/s)
  REAL(KIND = r8), INTENT(IN) :: par_z
  ! par absorbed per unit lai for canopy layer (w/m**2)
  REAL(KIND = r8), INTENT(IN) :: gb_mol
  ! leaf

In [6]:
print(python_function)

def ci_func(ci, lmr_z, par_z, gb_mol, je, cair, oair, rh_can, p, iv, c, atm2lnd_inst, photosyns_inst):
    # local variables
    ai = 0.0
    cs = 0.0
    term = 0.0
    aquad = 0.0
    bquad = 0.0
    cquad = 0.0
    r1 = 0.0
    r2 = 0.0

    forc_pbot = atm2lnd_inst.forc_pbot_downscaled_col
    c3flag = photosyns_inst.c3flag_patch
    ivt = patch.itype
    medlynslope = pftcon.medlynslope
    medlynintercept = pftcon.medlynintercept
    stomatalcond_mtd = photosyns_inst.stomatalcond_mtd
    ac = photosyns_inst.ac_patch
    aj = photosyns_inst.aj_patch
    ap = photosyns_inst.ap_patch
    ag = photosyns_inst.ag_patch
    an = photosyns_inst.an_patch
    vcmax_z = photosyns_inst.vcmax_z_patch
    cp = photosyns_inst.cp_patch
    kc = photosyns_inst.kc_patch
    ko = photosyns_inst.ko_patch
    qe = photosyns_inst.qe_patch
    tpu_z = photosyns_inst.tpu_z_patch
    kp_z = photosyns_inst.kp_z_patch
    bbb = photosyns_inst.bbb_patch
    mbb = photosyns_inst.mbb_patch

    if c3flag[p]:


In [24]:
import testing

In [25]:
import importlib
importlib.reload(testing)

<module 'testing' from '/home/ubuntu/code_conversion/testing.py'>

In [27]:

output = testing.run_tests(python_function, python_unit_tests, "python:3.8")

Pulling Docker image python:3.8...
Running tests in Docker container 8057c9e9adf24e5c1d18b577c0c93b3c39318c0eb0ea165fa54cd0b807682dcf...


In [28]:
print(output)

============================= test session starts ==============================

platform linux -- Python 3.8.17, pytest-7.3.2, pluggy-1.0.0
rootdir: /tests
collected 1 item                                                               

tmp2djiasty.py F                                                         [100%]

=================================== FAILURES ===================================
_________________________________ test_ci_func _________________________________

    def test_ci_func():
        # Define input variables
        ci = 400.0
        lmr_z = 1.0
        par_z = 100.0
        gb_mol = 0.1
        je = 10.0
        cair = 400.0
        oair = 210.0
        rh_can = 0.5
        p = 1
        iv = 1
        c = 1
    
        # Define expected output values
        expected_fval = 0.0
        expected_gs_mol = 0.0
    
        # Define variables for actual output values
        actual_fval = 0.0
        actual_gs_mol = 0.0
    
        # Call the function
>      

In [34]:
def generate_next_iteration(source_code, python_function, python_unit_tests, unit_test_output):
    messages = [
            {
            "role": "system",
            "content": """You're a programmer proficient in Fortran and Python. You can write and execute Python code by enclosing it in triple backticks, e.g. ```code goes here```""",
        },
        {
            "role": "user",
            "content": f"""Convert the following Fortran function to Python. ```
    {source_code}
    ```
    """
        },
        {
            "role": "assistant",
            "content": f"""Here's the converted Python function:

    ```python
    {python_function}
        ```
        """
        },
        {
            "role": "user",
            "content": f"""
            Here are some unit tests for the above code and the corresponding output.
            UNIT TESTS:
            ```python
    {python_unit_tests}
    ```
            RESPONSE:
            ```
            {unit_test_output}
            ```

            Modify the source code and/or unit tests to fix the bug.
            """
        }
    ]
    completion = llm.completion_with_backoff(
    model="gpt-3.5-turbo-16k-0613",
    messages=messages,
    temperature=0,
    )

    print(completion)

    response = completion.choices[0].message["content"]

    code = completion.choices[0].message["content"].split("```")[1]

    if 'pytest' in code:
        return response, 'test'


    return response, 'module'



In [33]:
res, part_changed = generate_next_iteration(source_code, python_function, python_unit_tests, output)
print(res)
print(part_changed)

{
  "id": "chatcmpl-7Tar4nqo15Ddvc9tDgKgyUYvIf61x",
  "object": "chat.completion",
  "created": 1687287538,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "It seems that there is an issue with the `atm2lnd_inst` and `photosyns_inst` variables in the unit test. They are assigned the value of `0.0`, which is causing the `AttributeError` when trying to access the attribute `forc_pbot_downscaled_col`. \n\nTo fix this, you need to provide the correct values for `atm2lnd_inst` and `photosyns_inst` in the unit test. Here's the modified unit test:\n\n```python\nimport pytest\n\ndef test_ci_func():\n    # Define input variables\n    ci = 400.0\n    lmr_z = 1.0\n    par_z = 100.0\n    gb_mol = 0.1\n    je = 10.0\n    cair = 400.0\n    oair = 210.0\n    rh_can = 0.5\n    p = 1\n    iv = 1\n    c = 1\n\n    # Define expected output values\n    expected_fval = 0.0\n    expected_gs_mol = 0.0\n\n    # Defi

In [37]:
updated_code = res.split("```")[1]
updated_code = updated_code.replace("python\n", "")

In [38]:
updated_code

'import pytest\n\ndef test_ci_func():\n    # Define input variables\n    ci = 400.0\n    lmr_z = 1.0\n    par_z = 100.0\n    gb_mol = 0.1\n    je = 10.0\n    cair = 400.0\n    oair = 210.0\n    rh_can = 0.5\n    p = 1\n    iv = 1\n    c = 1\n\n    # Define expected output values\n    expected_fval = 0.0\n    expected_gs_mol = 0.0\n\n    # Define variables for actual output values\n    actual_fval = 0.0\n    actual_gs_mol = 0.0\n\n    # Define dummy values for atm2lnd_inst and photosyns_inst\n    class DummyAtm2lndInst:\n        forc_pbot_downscaled_col = 0.0\n\n    class DummyPhotosynsInst:\n        c3flag_patch = [True]\n        stomatalcond_mtd = 0\n        ac_patch = [[0.0]]\n        aj_patch = [[0.0]]\n        ap_patch = [[0.0]]\n        ag_patch = [[0.0]]\n        an_patch = [[0.0]]\n        vcmax_z_patch = [[0.0]]\n        cp_patch = [0.0]\n        kc_patch = [0.0]\n        ko_patch = [0.0]\n        qe_patch = [0.0]\n        tpu_z_patch = [[0.0]]\n        kp_z_patch = [[0.0]]\n  

In [39]:
result = testing.run_tests(python_function, updated_code, "python:3.8")

Pulling Docker image python:3.8...
Running tests in Docker container 311a2f1fe79ac743e2936be147eeb2c158035def3c05a730f50fd143f5151da4...


In [41]:
print(result)

============================= test session starts ==============================

platform linux -- Python 3.8.17, pytest-7.3.2, pluggy-1.0.0
rootdir: /tests
collected 1 item                                                               

tmp5gnsa66y.py F                                                         [100%]

=================================== FAILURES ===================================
_________________________________ test_ci_func _________________________________

    def test_ci_func():
        # Define input variables
        ci = 400.0
        lmr_z = 1.0
        par_z = 100.0
        gb_mol = 0.1
        je = 10.0
        cair = 400.0
        oair = 210.0
        rh_can = 0.5
        p = 1
        iv = 1
        c = 1
    
        # Define expected output values
        expected_fval = 0.0
        expected_gs_mol = 0.0
    
        # Define variables for actual output values
        actual_fval = 0.0
        actual_gs_mol = 0.0
    
        # Define dummy values for a

In [42]:
new_result, part_changed = generate_next_iteration(source_code, python_function, updated_code, result)
print(new_result)

{
  "id": "chatcmpl-7Tb34ElgM0qbc1OyfOzp1TeBmbyHF",
  "object": "chat.completion",
  "created": 1687288282,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Apologies for the mistake. It seems that I made an error in the number of arguments passed to the `ci_func` function. I will correct it and provide the updated code and unit tests.\n\nHere's the updated code:\n\n```python\ndef ci_func(ci, lmr_z, par_z, gb_mol, je, cair, oair, rh_can, p, iv, c, fval, gs_mol, atm2lnd_inst, photosyns_inst):\n    # local variables\n    ai = 0.0\n    cs = 0.0\n    term = 0.0\n    aquad = 0.0\n    bquad = 0.0\n    cquad = 0.0\n    r1 = 0.0\n    r2 = 0.0\n\n    forc_pbot = atm2lnd_inst.forc_pbot_downscaled_col\n    c3flag = photosyns_inst.c3flag_patch\n    ivt = patch.itype\n    medlynslope = pftcon.medlynslope\n    medlynintercept = pftcon.medlynintercept\n    stomatalcond_mtd = photosyns_inst.stomatalcond_mtd\

In [43]:
test_result = testing.run_tests("""def ci_func(ci, lmr_z, par_z, gb_mol, je, cair, oair, rh_can, p, iv, c, fval, gs_mol, atm2lnd_inst, photosyns_inst):
    # local variables
    ai = 0.0
    cs = 0.0
    term = 0.0
    aquad = 0.0
    bquad = 0.0
    cquad = 0.0
    r1 = 0.0
    r2 = 0.0

    forc_pbot = atm2lnd_inst.forc_pbot_downscaled_col
    c3flag = photosyns_inst.c3flag_patch
    ivt = patch.itype
    medlynslope = pftcon.medlynslope
    medlynintercept = pftcon.medlynintercept
    stomatalcond_mtd = photosyns_inst.stomatalcond_mtd
    ac = photosyns_inst.ac_patch
    aj = photosyns_inst.aj_patch
    ap = photosyns_inst.ap_patch
    ag = photosyns_inst.ag_patch
    an = photosyns_inst.an_patch
    vcmax_z = photosyns_inst.vcmax_z_patch
    cp = photosyns_inst.cp_patch
    kc = photosyns_inst.kc_patch
    ko = photosyns_inst.ko_patch
    qe = photosyns_inst.qe_patch
    tpu_z = photosyns_inst.tpu_z_patch
    kp_z = photosyns_inst.kp_z_patch
    bbb = photosyns_inst.bbb_patch
    mbb = photosyns_inst.mbb_patch

    if c3flag[p]:
        # C3: Rubisco-limited photosynthesis
        ac[p, iv] = vcmax_z[p, iv] * max(ci - cp[p], 0.0) / (ci + kc[p] * (1.0 + oair / ko[p]))

        # C3: RuBP-limited photosynthesis
        aj[p, iv] = je * max(ci - cp[p], 0.0) / (4.0 * ci + 8.0 * cp[p])

        # C3: Product-limited photosynthesis
        ap[p, iv] = 3.0 * tpu_z[p, iv]
    else:
        # C4: Rubisco-limited photosynthesis
        ac[p, iv] = vcmax_z[p, iv]

        # C4: RuBP-limited photosynthesis
        aj[p, iv] = qe[p] * par_z * 4.6

        # C4: PEP carboxylase-limited (CO2-limited)
        ap[p, iv] = kp_z[p, iv] * max(ci, 0.0) / forc_pbot[c]

    # Gross photosynthesis. First co-limit ac and aj. Then co-limit ap
    aquad = params_inst.theta_cj[ivt[p]]
    bquad = - (ac[p, iv] + aj[p, iv])
    cquad = ac[p, iv] * aj[p, iv]
    quadratic(aquad, bquad, cquad, r1, r2)
    ai = min(r1, r2)

    aquad = params_inst.theta_ip
    bquad = - (ai + ap[p, iv])
    cquad = ai * ap[p, iv]
    quadratic(aquad, bquad, cquad, r1, r2)
    ag[p, iv] = max(0.0, min(r1, r2))

    # Net photosynthesis. Exit iteration if an < 0
    an[p, iv] = ag[p, iv] - lmr_z
    if an[p, iv] < 0.0:
        fval[0] = 0.0
        return

    # Quadratic gs_mol calculation with an known. Valid for an >= 0.
    # With an <= 0, then gs_mol = bbb or medlyn intercept
    cs = cair - 1.4 / gb_mol * an[p, iv] * forc_pbot[c]
    cs = max(cs, max_cs)
    if stomatalcond_mtd == stomatalcond_mtd_medlyn2011:
        term = 1.6 * an[p, iv] / (cs / forc_pbot[c] * 1.0E06)
        aquad = 1.0
        bquad = - (2.0 * (medlynintercept[patch.itype[p]] * 1.0E-06 + term) + (medlynslope[patch.itype[p]] * term) ** 2 / (gb_mol * 1.0E-06 * rh_can))
        cquad = medlynintercept[patch.itype[p]] * medlynintercept[patch.itype[p]] * 1.0E-12 + (2.0 * medlynintercept[patch.itype[p]] * 1.0E-06 + term * (1.0 - medlynslope[patch.itype[p]] * medlynslope[patch.itype[p]] / rh_can)) * term
        quadratic(aquad, bquad, cquad, r1, r2)
        gs_mol[0] = max(r1, r2) * 1.0E06
    elif stomatalcond_mtd == stomatalcond_mtd_bb1987:
        aquad = cs
        bquad = cs * (gb_mol - bbb[p]) - mbb[p] * an[p, iv] * forc_pbot[c]
        cquad = - gb_mol * (cs * bbb[p] + mbb[p] * an[p, iv] * forc_pbot[c] * rh_can)
        quadratic(aquad, bquad, cquad, r1, r2)
        gs_mol[0] = max(r1, r2)

    # Derive new estimate for ci
    fval[0] = ci - cair + an[p, iv] * forc_pbot[c] * (1.4 * gs_mol[0] + 1.6 * gb_mol) / (gb_mol * gs_mol[0])""", """import pytest

def test_ci_func():
    # Define input variables
    ci = 400.0
    lmr_z = 1.0
    par_z = 100.0
    gb_mol = 0.1
    je = 10.0
    cair = 400.0
    oair = 210.0
    rh_can = 0.5
    p = 1
    iv = 1
    c = 1

    # Define expected output values
    expected_fval = 0.0
    expected_gs_mol = 0.0

    # Define variables for actual output values
    actual_fval = [0.0]
    actual_gs_mol = [0.0]

    # Define dummy values for atm2lnd_inst and photosyns_inst
    class DummyAtm2lndInst:
        forc_pbot_downscaled_col = 0.0

    class DummyPhotosynsInst:
        c3flag_patch = [True]
        stomatalcond_mtd = 0
        ac_patch = [[0.0]]
        aj_patch = [[0.0]]
        ap_patch = [[0.0]]
        ag_patch = [[0.0]]
        an_patch = [[0.0]]
        vcmax_z_patch = [[0.0]]
        cp_patch = [0.0]
        kc_patch = [0.0]
        ko_patch = [0.0]
        qe_patch = [0.0]
        tpu_z_patch = [[0.0]]
        kp_z_patch = [[0.0]]
        bbb_patch = [0.0]
        mbb_patch = [0.0]

    # Create instances of dummy classes
    atm2lnd_inst = DummyAtm2lndInst()
    photosyns_inst = DummyPhotosynsInst()

    # Call the function
    ci_func(ci, lmr_z, par_z, gb_mol, je, cair, oair, rh_can, p, iv, c, actual_fval, actual_gs_mol, atm2lnd_inst, photosyns_inst)

    # Perform assertions
    assert expected_fval == actual_fval[0]
    assert expected_gs_mol == actual_gs_mol[0]""", "python:3.8")

Pulling Docker image python:3.8...
Running tests in Docker container cbe4f8375b590da95134c5928710459c3254744fb425845adffe09a48b28e236...


In [44]:
print(test_result)

============================= test session starts ==============================

platform linux -- Python 3.8.17, pytest-7.3.2, pluggy-1.0.0
rootdir: /tests
collected 1 item                                                               

tmpvaqsug8a.py F                                                         [100%]

=================================== FAILURES ===================================
_________________________________ test_ci_func _________________________________

    def test_ci_func():
        # Define input variables
        ci = 400.0
        lmr_z = 1.0
        par_z = 100.0
        gb_mol = 0.1
        je = 10.0
        cair = 400.0
        oair = 210.0
        rh_can = 0.5
        p = 1
        iv = 1
        c = 1
    
        # Define expected output values
        expected_fval = 0.0
        expected_gs_mol = 0.0
    
        # Define variables for actual output values
        actual_fval = [0.0]
        actual_gs_mol = [0.0]
    
        # Define dummy values f